## TODO
~~1. Batching!~~   
    ~~-Encoder.Forward의 input 모양 어떻게 되지? / .view 인자 확인!~~
2. Attention  
3. Teacher Forcing  
~~4. Parameter(things to be updated) 등록 잘 됐나 확인(= 그래프 잘 그린 건지 어케 확인하나?)~~
5. Train / Dev 사전에 나누기
6. Pretrained word vector을 쓸 수가 있나..?  
** SOS, EOS, PAD token 관리 어디서?**   
** INTO GPU: x batch, y batch, parameter On GPU**
7. model save등등 train 뒷단 얘기  
    -torch.save


#### NOTE
1. Decoder가 2 layer일때, initial hidden?  
    - https://discuss.pytorch.org/t/understanding-output-of-lstm/12320/2
    - hidden 의 dimension
2. embedding 거친 후엔 batch_size x seq_len x embedding_dim 임!
    - embedding input은 batch_size x seq_len
2. LSTM의 batch_first
3. LSTM input의 dimension
4. NLLLoss()의 dimension

In [1]:
from build_vocab import *

In [3]:
meta = build_meta()
tagVocab, rvVocab = filter_by_cnt(2, 2, 'data/products.pkl')
data = prepareData(meta, tagVocab, rvVocab, 'data/products.pkl')

Building vocab for tags ...
Building vocab for reviews ...
3339 out of 3465 words left, which is 96.36363636363636 %
21036 out of 37012 words left, which is 56.83562087971469 %


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from tqdm import tqdm_notebook

In [5]:
class Config():
    dropout = 0.2
    batch_size = 2
    num_steps = 10
    
    # Encoder
    rating_size = len(meta.rating2idx)
    category_size = len(meta.lowcat2idx)
    tag_size = len(tagVocab.word2idx)    # tag vocab_size
    pretrained = False 
    attribute_size = 64
    hidden_size = 512 # fixed-vector size 
    
    # Decoder
    num_layers = 2
    output_size = len(rvVocab.word2idx)    # review vocab_size
    
    # TEMP - 이거 어디서 관리하지? 여긴 아닌뎅
    padding_idx = 0

In [6]:
class Encoder(nn.Module):
    def __init__(self, config):        
        super().__init__()
        self.config = config
        
        self.emb_rating = nn.Embedding(self.config.rating_size, self.config.attribute_size)   
        self.emb_category = nn.Embedding(self.config.category_size, self.config.attribute_size)
        self.emb_tag = nn.Embedding(self.config.tag_size, self.config.attribute_size,
                                   padding_idx=self.config.padding_idx)        
        self.out = nn.Linear(self.config.attribute_size * 3, self.config.hidden_size*self.config.num_layers)
        self.init_hidden()
        
    def forward(self, rating, category, tag):
        """
        Inputs:
            rating: TENSOR of shape (batch_size, 1)
            category: TENSOR of shape (batch_size, 1)
            tag : 1) TENSOR of shape (batch_size, tag_MAXLEN)
        Returns:
            concatenated attr for attention, encoder_output
        """
        
        assert len(rating) == len(category) == len(tag)
        attr_rating = self.emb_rating(rating)    # view no need!
        attr_category = self.emb_category(category)
        tag_len = self.get_tag_len(tag)    
        attr_tag = torch.sum(self.emb_tag(tag), 1, keepdim=True) / tag_len    # CBOW
                # 여기서 이렇게 해도 backprop 맞게 되나??
        
        attr = torch.cat((attr_rating, attr_category, attr_tag), 2)
        out = self.out(attr)
        encoder_output = F.tanh(out)
        return attr, encoder_output
    
    def get_tag_len(self, tag): 
        """padding 제외한 token 개수"""
        return torch.sum(tag!=self.config.padding_idx, 1).unsqueeze(1).unsqueeze(1).type(torch.float)
        
    def init_hidden(self):
        for param in self.parameters():
            nn.init.uniform_(param, -0.08, 0.08)

In [7]:
print("===== Testing encoder... with single input =====")
config = Config()
encoder = Encoder(config)
rating = torch.tensor([[3]]).type(torch.long)    # idx of rating in tensor
category = torch.tensor([[7]]).type(torch.long)  # idx of category in tensor
tag = torch.tensor([[1,2,1]]).type(torch.long)    # idxs of tag
attr, encoder_output = encoder(rating,category,tag)
print(attr.size())
print(encoder_output.size())

print("\n===== with multiple inputs =====")
rating = torch.tensor([[3],[4]])
category = torch.tensor([[8],[1]])
tag = torch.tensor([[1,2,1,2], [0,1,1,0]])    # 이렇게 하려면 padding 되어 있어야해!
attr, encoder_output = encoder(rating,category,tag)
print(attr.size())
print(encoder_output.size())

===== Testing encoder... with single input =====
torch.Size([1, 1, 192])
torch.Size([1, 1, 1024])

===== with multiple inputs =====
torch.Size([2, 1, 192])
torch.Size([2, 1, 1024])


In [8]:
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # TODO: if self.config.pretrained = True
        self.embedding = nn.Embedding(self.config.output_size, self.config.hidden_size)
        self.lstm = nn.LSTM(self.config.hidden_size, self.config.hidden_size, \
                            num_layers=self.config.num_layers, dropout=self.config.dropout, \
                           batch_first=True)
        self.out = nn.Linear(self.config.hidden_size, self.config.output_size)
        
    def forward(self, input_token, hidden):
        """
        Inputs:
            input_token: TENSOR of shape (batch_size, 1)
            hidden: from last hidden of encoder
        Returns:
        """
        # 가운데 1이니까 unroll방식으로만!
        output = self.embedding(input_token)
        # LSTM의 hidden은 (hx, cx)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = F.log_softmax(output, dim=2)
        return output, hidden

    def initHidden(self):
        pass

In [9]:
print("====testing decoder with encoder_output...(batch=2)====\n")
decoder = Decoder(config)
# h_0을 초기화시키고, c_0은 냅두기!!!(0으로 하든가 임의값으로 하든가)
batch_size = encoder_output.size(0)    # same as len(encoder_output) and 
h_ = encoder_output.view(config.num_layers, 2, config.hidden_size)    # 2는 batch_size
c_ = torch.zeros_like(h_)
hidden = (h_, c_)

input_token = torch.tensor([[1],[2]])  # shape (batch_size, seq_len)
output, hidden = decoder(input_token, hidden)
print("input_token.size(): ", input_token.size())
print("hidden[0].size(): ", hidden[0].size())
print("hidden[1].size(): ", hidden[1].size())
print("output.size(): ", output.size())



====testing decoder with encoder_output...(batch=2)====

input_token.size():  torch.Size([2, 1])
hidden[0].size():  torch.Size([2, 2, 512])
hidden[1].size():  torch.Size([2, 2, 512])
output.size():  torch.Size([2, 1, 21038])


In [10]:
# def splitHidden(encoder_output, config):
#     h_0 = encoder_output.view(config.num_layers, encoder_output.size(0), \
#                               config.hidden_size)
#     c_0 = torch.zeros_like(h_0) 
#     return (h_0, c_0)

# def train(encoder, decoder, data, loss_fn, optimizer, batch_size, num_steps, print_every, verbose=False):
#     encoder.train()
#     decoder.train()

#     for t in tqdm_notebook(range(num_steps)):
#         optimizer.zero_grad()
#         rating_tensor, category_tensor, tag_tensor, target_tensor = data_iterator(data, batch_size)
#         target_length = target_tensor.size(-1)
        
#         attr, encoder_output = encoder(rating_tensor, category_tensor, tag_tensor)
#         decoder_hidden = splitHidden(encoder_output, encoder.config)        
        
#         decoder_input = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
#         loss = 0
#         for idx in range(target_length): 
#             decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)            
#             topv, topi = decoder_output.topk(1)
#             decoder_input = topi.detach().view(batch_size, 1)
#             loss += loss_fn(decoder_output.squeeze(), target_tensor[:,idx])
#         num_actual_token = torch.sum(target_tensor != encoder.config.padding_idx).item()
#         loss /= num_actual_token
        
#         if t % print_every == 0:
#             print("loss at %d step: %f" % (t, loss))
#         loss.backward()
# #         nn.utils.clip_grad_norm_(encoder.parameters(), 0.25)
# #         nn.utils.clip_grad_norm_(encoder.parameters(), 0.25)
#         optimizer.step()     

In [11]:


def splitHidden(encoder_output, config):
    h_0 = encoder_output.view(config.num_layers, encoder_output.size(0), \
                              config.hidden_size)
    c_0 = torch.zeros_like(h_0) 
    return (h_0, c_0)

def train(encoder, decoder, data, loss_fn, optimizer, batch_size, num_steps, print_every, verbose=False):
    encoder.train()
    decoder.train()

    for t in tqdm_notebook(range(num_steps)):
        optimizer.zero_grad()
        rating_tensor, category_tensor, tag_tensor, target_tensor = data_iterator(data, batch_size)
        target_length = target_tensor.size(-1)
        
        attr, encoder_output = encoder(rating_tensor, category_tensor, tag_tensor)
        decoder_hidden = splitHidden(encoder_output, encoder.config)        
        
        decoder_input = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        decoder_outputs = []
        for idx in range(target_length): 
            
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach().view(batch_size, 1)
            decoder_outputs.append(decoder_output)

        # 이 아래 두 개 shape 조정은 loss function 요구 사항 맞추기 위함!
        decoder_outputs = torch.cat(decoder_outputs, 1).view(batch_size*target_length, -1)
        target_tensor = target_tensor.view(-1)
        loss = loss_fn(decoder_outputs, target_tensor) 
        num_actual_token = torch.sum(target_tensor != encoder.config.padding_idx).item()
        loss /= num_actual_token
        
        if t % print_every == 0:
            print("loss at %d step: %f" % (t, loss))

            
        loss.backward()
        optimizer.step()     



In [12]:
config = Config()
encoder = Encoder(config)
decoder = Decoder(config)
# data is defined on top
params = list(encoder.parameters()) + list(decoder.parameters())
loss_fn = nn.NLLLoss(size_average=False, ignore_index=config.padding_idx)
optimizer = optim.Adam(params, lr=0.002)
#train(encoder, decoder, data, loss_fn, optimizer, config.batch_size, config.num_steps)
train(encoder, decoder, data, loss_fn, optimizer, 2, 1000, 1, verbose=True)

loss at 0 step: 9.952996
loss at 1 step: 9.942863
loss at 2 step: 9.853773
loss at 3 step: 9.812474
loss at 4 step: 9.472528
loss at 5 step: 8.640425
loss at 6 step: 9.127494
loss at 7 step: 6.658671
loss at 8 step: 7.340724
loss at 9 step: 7.641273
loss at 10 step: 9.708379
loss at 11 step: 8.097731
loss at 12 step: 8.834807
loss at 13 step: 8.271302
loss at 14 step: 8.786379
loss at 15 step: 8.618076


KeyboardInterrupt: 

In [16]:
class Attr2Seq(nn.Module):
    def __init__(self, config, criterion):
        super().__init__()
        self.config = config
        self.criterion = criterion
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
    
    def forward(self, rating, category, tag, target_tensor):
        # 함수 호출시 *[rating, category, tag]하기!

        batch_size = target_tensor.size(0)
        target_length = target_tensor.size(-1)
        attr, encoder_output = self.encoder(rating,category,tag)
        batch_size = encoder_output.size(0)
        decoder_hidden = self.splitHidden(encoder_output)
        input_token = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        
        decoder_outputs = []
        for idx in range(target_length): 
            decoder_output, decoder_hidden = decoder(input_token, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            input_token = topi.detach().view(batch_size, 1)
            decoder_outputs.append(decoder_output)

        # 이 아래 두 개 shape 조정은 loss function 요구 사항 맞추기 위함!
        decoder_outputs = torch.cat(decoder_outputs, 1).view(batch_size*target_length, -1)
        target_tensor = target_tensor.view(-1)
        loss = self.criterion(decoder_outputs, target_tensor) #/ batch_size
                        # batch_size로 나눠주기
#         print("decoder_outputs: ", decoder_outputs)
#         print("target_tensor: ", target_tensor)
        return loss
    
    def splitHidden(self, encoder_output):
        h_0 = encoder_output.view(self.config.num_layers, encoder_output.size(0), \
                                  self.config.hidden_size)
        c_0 = torch.zeros_like(h_0) 
        return (h_0, c_0)
    
    def inference(self):
        pass


In [17]:
print("==== testing Attr2Seq model... ====")
criterion = nn.NLLLoss()     # TODO: padding loss빼주기ㅠㅠ
model = Attr2Seq(config, criterion)
input_list = [torch.tensor([[2]]).type(torch.long), torch.tensor([[7]]).type(torch.long),
              torch.tensor([[0,2,1]])]
target_tensor = torch.tensor([[4]])
loss = model(*input_list, target_tensor)
print(loss)

print("\n==== baching(size 2) ====")
input_list = [torch.tensor([[2],[1]]).type(torch.long), torch.tensor([[7],[5]]).type(torch.long),
              torch.tensor([[0,2,1],[2,0,1]])]
target_tensor = torch.tensor([[9,2,1,3,5], [3,2,3,2,1]])
loss = model(*input_list, target_tensor)
print(loss)

==== testing Attr2Seq model... ====
tensor(10.0135)

==== baching(size 2) ====
tensor(10.0085)


In [54]:
# 그 전에 Train / Dev 나누기
# TODO: data랑 받아서 정의된 data iterator로 불러오기

def train(data, model, optimizer, num_steps):
    model.train()

    for t in tqdm_notebook(range(num_steps)):
#        a = list(model.parameters())[0]
        optimizer.zero_grad()
        r, c, t, rv = data_iterator(data, batch_size = 2)
        loss = model(r, c, t, rv)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()        
#         for name, param in model.named_parameters():
#             print(name)
#             print(param.grad)
#        b = list(model.parameters())[0]
#        print(loss)
    return loss.item()# / target_length

In [55]:
model = Attr2Seq(config, criterion)
optimizer = optim.SGD(model.parameters(), lr=1)
#optimizer = optim.RMSprop(model.parameters(), lr=0.2, alpha=0.95)
temp_loss = train(Encoded, model, optimizer, 2)
print(temp_loss)


nan


In [101]:
torch.cuda.device_count()

2

In [104]:
torch.cuda.is_available()

True

In [56]:
class Attr2Seq(nn.Module):
    def __init__(self, config, criterion):
        super().__init__()
        self.config = config
        self.criterion = criterion
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
    
    def forward(self, rating, category, tag, target_tensor):

        batch_size = target_tensor.size(0)
        target_length = target_tensor.size(-1)
        attr, encoder_output = self.encoder(rating,category,tag)
        batch_size = encoder_output.size(0)
        decoder_hidden = self.splitHidden(encoder_output)
        input_token = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        
        loss = 0
        for idx in range(target_length): 
            decoder_output, decoder_hidden = decoder(input_token, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            input_token = topi.detach().view(batch_size, 1)
            loss += self.criterion(decoder_output.squeeze(), target_tensor[:,idx].view(-1))

        loss = loss / batch_size
        return loss
    
    def splitHidden(self, encoder_output):
        h_0 = encoder_output.view(self.config.num_layers, encoder_output.size(0), \
                                  self.config.hidden_size)
        c_0 = torch.zeros_like(h_0) 
        return (h_0, c_0)
    
    def inference(self):
        pass


In [60]:
# 그 전에 Train / Dev 나누기
# TODO: data랑 받아서 정의된 data iterator로 불러오기

def train(data, model, criterion, optimizer, num_steps):
    model.train()
    loss_fn = criterion
    for t in tqdm_notebook(range(num_steps)):
        a = list(model.parameters())[0]
        optimizer.zero_grad()
        r, c, t, rv = data_iterator(data, batch_size = 2)
            
        loss = model(r, c, t, rv)
#        decoder_output = model(r, c, t, rv)
#        rv = rv.view(-1)
#        loss = loss_fn(decoder_output, rv)
        loss.backward()
#        print(loss.grad_fn)
        print(loss)
        
        #nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()        
        for name, param in model.named_parameters():
            print(name)
            print(param.grad)
        b = list(model.parameters())[0]
        
        print(loss)
    return loss.item()# / target_length

In [61]:
criterion = nn.NLLLoss()
model = Attr2Seq(config, criterion)
params = list(model.encoder.parameters()) + list(model.decoder.parameters())
#optimizer = optim.SGD(model.parameters(), lr=1)
optimizer = optim.SGD(params, lr=1)

#optimizer = optim.RMSprop(model.parameters(), lr=0.2, alpha=0.95)
temp_loss = train(Encoded, model, criterion, optimizer, 10)
print(temp_loss)

Exception in thread Thread-22:
Traceback (most recent call last):
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



tensor(239.8178)
encoder.emb_rating.weight
tensor(1.00000e-03 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,

tensor(55.0357)
encoder.emb_rating.weight
tensor(1.00000e-03 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 

tensor([[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]])
encoder.out.weight
tensor([[nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        ...,
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.]])
encoder.out.bias
tensor([nan., nan., nan.,  ..., nan., nan., nan.])
decoder.embedding.weight
None
decoder.lstm.weight_ih_l0
None
decoder.lstm.weight_hh_l0
None
decoder.lstm.bias_ih_l0
None
decoder.lstm.bias_hh_l0
None
decoder.lstm.weight_ih_l1
None
decoder.lstm.weight_hh_l1
None
decoder.lstm.bias_ih_l1
None
decoder.lstm.bias_hh

tensor([[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]])
encoder.out.weight



KeyboardInterrupt: 